# Configuração final da estrutura de diretórios

* data_ex (catalog) 
    * metadados (schema)
        * meta_bronze (table)
        * meta_silver (table)
        * meta_gold (table)
    * lhdw (schema) 
        * versao* (volumes)       
    * bronze (schema)
        * versao* (volume)
            * bronze_dim_categoria_produto
            * bronze_dim_cliente 
            * bronze_dim_data 
            * bronze_dim_localidade
            * bronze_dim_produto 
            * bronze_fato_vendas  
    * silver (schema)
        * versao* (volume)
            * silver_dim_categoria_produto
            * silver_dim_cliente 
            * silver_dim_data 
            * silver_dim_localidade
            * silver_dim_produto 
            * silver_fato_vendas
    * gold  (schema)
        * versao* (volume)
            * dim_categoria_produto
            * dim_cliente 
            * dim_data 
            * dim_localidade
            * dim_produto 
            * fato_vendas

Tornar o escrito a cima em um diagrama para ser um dos anexos ao storytelling

# Registro de Logs de Carga
* id_carga (string (uuid))
* id_job (string (uuid))
* nome_arquivo (string)
* fonte (string)
* camada (string)
* path_destino (string)
* data_inicio (utc_timestamp)
* data_fim (utc_timestamp)
* duracao_ms (bigint)
* registros_lidos (bigint)
* registros_gravados (bigint)
* status (string)
* mensagem_erro (string)
* data_execucao (date(yyyy-MM-dd))

## Exemplo
| Campo                  | Valor                                         |
| ---------------------- | --------------------------------------------- |
| **id_carga**           | `8f2c3b9e-4d7a-4e91-b0c1-9a5f7c6d2e11`        |
| **id_job**             | `1a6b4c20-3c77-4c0e-9f5a-8b12f9e3a001`        |
| **nome_arquivo**       | `cliente2.csv`                                |
| **fonte**              | `FILESYSTEM_LOCAL`                            |
| **camada**             | `bronze`                                      |
| **path_destino**       | `/Volumes/workspace/bronze/clientes/cliente2` |
| **data_inicio**        | `2026-02-01 10:15:03.421 UTC`                 |
| **data_fim**           | `2026-02-01 10:15:05.987 UTC`                 |
| **duracao_ms**         | `2566`                                        |
| **registros_lidos**    | `12450`                                       |
| **registros_gravados** | `12450`                                       |
| **status**             | `SUCCESS`                                     |
| **mensagem_erro**      | `NULL`                                        |
| **data_execucao**      | `2026-02-01`                                  |


In [0]:
# Criação do catalog
spark.sql("create catalog if not exists data_ex")

# Criação do schema
spark.sql("create schema if not exists data_ex.metadados")
spark.sql("create schema if not exists data_ex.lhdw")
spark.sql("create schema if not exists data_ex.bronze")
spark.sql("create schema if not exists data_ex.silver")
spark.sql("create schema if not exists data_ex.gold")

# Criação das tabelas de metadados fixos
spark.sql("""create table if not exists data_ex.metadados.meta_bronze (
    id_job string
) using delta """)

spark.sql("""create table if not exists data_ex.metadados.meta_silver (
    id_job string
) using delta """)

spark.sql("""create table if not exists data_ex.metadados.meta_gold  (
    id_job string
) using delta """)

In [0]:
%skip
#spark.sql("drop schema if  exists data_ex.lhdw cascade")
#spark.sql("drop schema if  exists data_ex.bronze cascade")
spark.sql("drop schema if  exists data_ex.metadados cascade")
#spark.sql("drop schema if  exists data_ex.silver cascade")
#spark.sql("drop schema if  exists data_ex.gold cascade")
# spark.sql("drop table if exists data_ex.metadados.meta_bronze")
# spark.sql("drop table if exists data_ex.metadados.meta_silver")
# spark.sql("drop table if exists data_ex.metadados.meta_gold")

## Inserindo metadados

In [0]:
import uuid

id_job_bronze = str(uuid.uuid4())
id_job_silver = str(uuid.uuid4())
id_job_gold = str(uuid.uuid4())

spark.sql(f"insert into data_ex.metadados.meta_bronze values ('{id_job_bronze}')")
spark.sql(f"insert into data_ex.metadados.meta_silver values ('{id_job_silver}')")
spark.sql(f"insert into data_ex.metadados.meta_gold values ('{id_job_gold}')")





In [0]:
spark.sql("select * from data_ex.metadados.meta_bronze").show()

# Importação pelo Github

### Função de download

In [0]:
from pyspark.sql.functions import *
try:
   #volumes = spark.sql("""select count(volumes) from data_ex.lhdw""")
   volumes = spark.sql("show volumes in data_ex.lhdw")
   print(volumes.show())
except Exception as e:
    print("Não foi possível acessar o schema data_ex.lhdw")

In [0]:
import urllib.request

# Função para baixar os arquivos
def download_dataset(csv_files, url, lhdw_path):
    # Tenta realizar o download dos arquivos
    try :
        # Variavel para armazenar o nome do novo volume
        volume = ""
        try:
            # Conta quantos volumes já existem dentro desse do schema
            numero = spark.sql("show volumes in data_ex.lhdw").count()
            # Cria o comando sql para criar um novo volume
            volume = f"download{numero+1:03d}"
            sql_command = f"create volume if not exists data_ex.lhdw.{volume}"
            # Executa o comando sql acima
            spark.sql(sql_command)
        # lanca um execao caso houver um erro
        except Exception as e:
            print(f"Erro ao criar o volume: {e}")
        for file in csv_files:
            down_path = f"{url+file}"
            up_path = f"{lhdw_path+volume}/{file}"
            urllib.request.urlretrieve(down_path, up_path)
            print(f"Csv {file} baixado com sucesso!")
    # Caso não consigua realizar o download, uma exceção é lançada
    except Exception as e:
        print(f"Erro ao baixar o arquivo: {file} - {e}")
    return volume

### Utilizaçao da função de download

In [0]:
# Lista de .csv
csv_files = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

# Reposotório do dataset para o desafio
url = "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/"

# Path para o schema de armazenamento, sem o volume
lhdw_path = "/Volumes/data_ex/lhdw/"

volume_salvo = download_dataset(csv_files, url, lhdw_path)
print(volume_salvo)

# Bronze Layer

## Configuração do servidor para trabalho no Bronze Layer

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Load Data Bronze") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados


## Trabalhos no Bronze Layer

#### Variáveis para leitura

In [0]:
# Importação das bibliotecas/funções necessárias
# from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType

# Definição dos diretorios de trabalho
lhdw_path = f"/Volumes/data_ex/lhdw/{volume_salvo}"
sql_command = f"create volume if not exists data_ex.bronze.{volume_salvo}"
# Criação do schema bronze
spark.sql(sql_command)
# Definição do path bronze
bronze_path = f"/Volumes/data_ex/bronze/{volume_salvo}"

# Arquivos csv a serem trabalhados

csv_files = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

# Lista dos nomes dos schemas em ordem de execução
schemas = []


# Definição manual do schemas a serem aplicados nos arquivos csv (futuros parquets)
schema_categoria = StructType([
    StructField("categoria_id", LongType(), True),
    StructField("categoria_nome", StringType(), True),
])
schemas.append(schema_categoria)

schema_cliente = StructType([
    StructField("cliente_id", LongType(), True),
    StructField("nome_cliente", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True)
])
schemas.append(schema_cliente)

schema_data = StructType([
    StructField("data_id", LongType(), True),
    StructField("data", DateType(), True),
    StructField("ano", IntegerType(), True),
    StructField("mes", IntegerType(), True),
    StructField("dia", IntegerType(), True),
    StructField("dia_semana", StringType(), True), 
    StructField("final_de_semana", BooleanType(), True)
])
schemas.append(schema_data)

schema_localidade = StructType([
    StructField("localidade_id", LongType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True),
    ])  
schemas.append(schema_localidade)

schema_produto = StructType([
    StructField("produto_id", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("categoria_nome", StringType(), True)
])
schemas.append(schema_produto)

schema_vendas = StructType([
    StructField("venda_id", LongType(), True),
    StructField("cliente_id", LongType(), True),
    StructField("produto_id", LongType(), True),
    StructField("data_id", LongType(), True),
    StructField("categoria_id", LongType(), True),
    StructField("localidade_id", LongType(), True),
    StructField("quantidade", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("valor_total", DoubleType(), True)
])
schemas.append(schema_vendas)



#### Função de Leitura

In [0]:
# Criando uma lista para armazenar os Sparks DataFrames
spark_df = []

# Criação do Spark DataFrame, leitura dos csv, aplicação dos schemas e escrita em parquet 
def bronze_read(csv_files, schemas):
    for file, schema in zip(csv_files, schemas):
        # Tenta ler os arquivos csv em lhdw_path
        try:
            # Se schema não for vendas, aplica o schema correspondente a file
            if schema != schema_vendas:
                df = spark.read\
                    .option("header", "true")\
                    .schema(schema)\
                    .csv(f"{lhdw_path}/{file}")
                print(f"Leitura do arquivo {file} bem sucedida.")
                spark_df.append(df)

            # Se for vendas, deve repetir a leirura do arquivo n vezes
            else:
                # Tenta ler o caminho no qual vendas esta
                try:
                    paths = dbutils.fs.ls(lhdw_path)
                    # Conta o numero csv vendas existentes
                    n = len([n for n in paths if n.name.startswith("vendas")])        
                    # Aplica um loop para ler todos os arquivos vendas
                    for i in range(1, n+1):
                        try:
                            df = spark.read.option("header", "true")\
                                .schema(schema_vendas)\
                                .csv(f"{lhdw_path}/vendas_part{i}.csv")
                            spark_df.append(df)
                            print(f"Leitura do arquivo vendas_part{i}.csv bem sucedida.")
                           
                        except Exception as e:
                            print(f"Erro ao ler o arquivo {lhdw_path}/vendas_part{i}.csv: {e}")
                except Exception as e:
                    print("ERRO")
                    print(f"Erro ao ler o caminho no qual no qual vendas esta {e}")
            
        except Exception as e:
            print(f"Erro ao ler o arquivo {file}: {e}")


bronze_read(csv_files, schemas)

## Salvar no Bronze Layer

In [0]:
# Função para salvar os arquivos na camada bronze
def save(spark_df, camada, dime, fato, n_fatos):
    # Variaveis auxiliares
    n_dims = len(dime)# conta o numero de dimensões 
    indice_fato = 0              
    
    # Loop para criar as dimensões
    for i in range(n_dims):
        try:
            spark_df[i].write.format("delta").mode("overwrite").save(f"{camada}/{dime[i]}")
            indice_fato = indice_fato + 1
            print(f"DataFrame salvo com sucesso em {camada}/{dime[i]}")
        except Exception as e:
            print(f"Erro ao salvar o dataframe: {e}")

    #display(spark_df[indice_fato])
    #Loop para criar a tabela de fato vendas
    for i in range(0, n_fatos):
        try:
            spark_df[indice_fato].write.format("delta").mode("append").save(f"{camada}/{fato}")
            indice_fato = indice_fato + 1
            print(f"DataFrame salvo com sucesso em {camada}/{fato}")
        except Exception as e:
            print(f"Erro ao salvar o dataframe vendas: {e}")



In [0]:
# Definição do nomes das dimensões a serem salvas
dim = [
  "bronze_dim_categoria_produto",
  "bronze_dim_cliente",
  "bronze_dim_data", 
  "bronze_dim_localidade",
  "bronze_dim_produto", 
]
fato = "bronze_fato_vendas"

n_fatos = 4

# Chamada da função de salvamento
save(spark_df, bronze_path, dim, fato, n_fatos)

In [0]:
df = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_fato_vendas").count()
print(df)

## Limpar memória

In [0]:
import gc

gc.collect()